## Process WikiCorp Dataset to have a large vocab
    * Use this for comparing word embeddings

In [1]:
import inflect,os,sys,re
from utils.sparse_utils import saveSparseHDF5
from utils.misc import savePickle
p = inflect.engine()

In [2]:
import numpy as np
from utils.sparse_utils import loadSparseHDF5

In [3]:
from load import loadDataset
dset1 = loadDataset('scws')
dset2 = loadDataset('wordsim353')
dset3 = loadDataset('imdb')
dset4 = loadDataset('sst_fine')
dset5 = loadDataset('rotten_tomatoes')
dset6 = loadDataset('sst_binary')
def wParse(k):
    ws = re.sub('[^\w\s]','', k.strip())
    ws = ws.lower()
    return ws
vocab_emb = set([wParse(dset1[k]['w1']) for k in dset1]+[wParse(dset1[k]['w2']) for k in dset1]
               +[wParse(k) for k in dset3['word2idx'].keys()]+[wParse(k) for k in dset4['word2idx'].keys()]
               +[wParse(k) for k in dset5['word2idx'].keys()])
print len(vocab_emb),' words needed'

Loading...
Read  1  objects
Read  1  objects
Read  1  objects
Read  1  objects
31592  words needed


In [4]:
#Load wikicorp vocab
assert os.path.exists('./wikicorp/WestburyLab.wikicorp.201004-split_hyphen.feat'),'Feature file not found'
with open('./wikicorp/WestburyLab.wikicorp.201004-split_hyphen.feat','r') as f:
    vocab = [k.strip().split(' ')[0] for k in f.readlines()]
print len(vocab)
vocab_arr = np.array(vocab)
vocabset   = set(vocab)

2001133


In [5]:
#The number of singular nouns (flags->flag)
vlist  = []
w2idx  = {}

for idx,v in enumerate(vocab):
    sv = p.singular_noun(v)
    if sv:
        vlist.append(sv)
        w2idx[sv] = idx
    else:
        vlist.append(v)
        w2idx[v]  = idx
    
vocab_singular_only = set(vlist)
vocab_singular_list = np.array(vlist)
print vocab_singular_list.shape

(2001133,)


In [6]:
word2idx_main     = {}
for i,w in enumerate(vocab):
    word2idx_main[w] = i

word2idx_singular = {}
for i,w in enumerate(vlist):
    word2idx_singular[w]=i

In [7]:
#Find all the words they map to
from_vocab = []
from_vocab_singular = []
absent     = []

for w in vocab_emb:
    if w in vocabset:
        #idxlst_vocab = np.where(w==vocab_arr)[0].tolist()
        #idxlst_vocab = [vocab.index(w)]
        #assert len(idxlst_vocab)==1,'nsd'
        idx = word2idx_main[w]
        from_vocab  += [idx]
    elif w in vocab_singular_only:
        #idxlst = np.where(w==vocab_singular_list)[0].tolist()
        #assert len(idxlst)==1,'nsd'
        #idxlst = [vlist.index(w)]
        idx  = word2idx_singular[w]
        from_vocab_singular+=[idx]
    else:
        print w,
        absent.append(w)
print ''

strainingly sailboaters addessi cipherlike demencial entretenida realizando colocar splatterfests newtonwanders prechewed melodramticos efteriades fizzability originalidad psychodramatics shayamalan emocionalmente camareras seldahls flmica unhibited aholas vidgame perseguio animaton drippiness daneses badbut actingand smnes characteris overstylized argumentao pasachke spittingly nuld margolo disfrutable watstein movieif greasiest problemits shimmeringly spinottis mediocridade reeboir movieyou egocentricities hlots roisterous bruckheimeresque uzumakis descompromissado sureif shagster stuffiest preocupar actorliness expresar hotsies unsuspenseful butterfingered brilhantismo premisa imaxy enfrentados musclefest cassavettes precisamente outgag ourside acabamos decirles unclassifiably tanovics telanovela choquarts stepdads djeinaba formuliac brehenys inquestionvel covardia nonchallenging emotiva fascinantes surfacey imponderably motherinlaw dirigir dorkier petins runteldat choquart dognini 

In [8]:
print len(set(from_vocab))
print len(set(from_vocab_singular))
print len(set(from_vocab+from_vocab_singular))
idx_to_preserve = from_vocab+from_vocab_singular
print 'Preserving ',len(idx_to_preserve),' words from original vocab'

30865
300
31084
Preserving  31165  words from original vocab


In [9]:
data = loadSparseHDF5('dataset','./wikicorp/WestburyLab.wikicorp.201004-split_hyphen.h5')
counts = np.array(data.sum(0)).squeeze().astype(int)
print counts.shape

(2001133,)


In [10]:
preserve_counts = [counts[k] for k in idx_to_preserve]
print 'See how frequent the words are: ',preserve_counts

See how frequent the words are:  [1254, 27289, 112, 6730, 12767, 8055, 784, 154, 830, 254, 2485, 654, 3194, 4937, 5441, 902, 900, 35326, 14814, 163, 581, 2819, 21, 4256, 8748, 380, 2884, 2104, 1004, 1246, 14380, 297, 2271, 14454, 185, 55, 10024, 262, 772, 2321, 973, 307, 24, 1201, 3723, 153421, 270, 28824, 1258, 29679, 1018, 8222, 52569, 2819, 2042, 378, 2119, 74080, 466, 65, 13003, 2203, 29943, 1967, 1193, 3, 1380, 608282, 1257, 1666, 274, 27162, 35, 70, 5062, 1859, 72, 8974, 347, 958, 67, 8279, 2781, 12274, 4360, 680, 509, 543, 85, 18564, 1767, 1046, 1952, 29, 59308, 19198, 2587, 76946, 1222, 49, 4446, 410, 1592, 1445, 1425, 2980, 6151, 27606, 8507, 105, 566, 194, 13, 438, 246, 158, 63, 9464, 354, 668, 25, 7350, 1974, 131, 30612, 35232, 8310, 392, 728, 103882, 11908, 5037, 565, 21, 26307, 2929, 84659, 21394, 3078, 1140, 107, 7597, 395, 12816, 2662, 16904, 10681, 355, 14256, 3675, 233, 2341, 104, 18165, 18540, 6622, 1276, 6163, 3811, 17270, 17547, 5952, 430, 9064, 31, 6720, 66019, 235

In [11]:
print np.sort(preserve_counts)[995:1000]

[21 22 22 22 22]


In [12]:
MAXVOCAB   = 35000
sorted_idx = list(set(np.argsort(counts)[-MAXVOCAB:].tolist() + idx_to_preserve))
print np.sort(counts[sorted_idx])

print  len(sorted_idx),np.max(sorted_idx),np.min(sorted_idx),len(vocab)
#Add vectors corresponding to embedding words

[      2       2       2 ..., 1170157 1341684 1949093]
46379 1999964 126 2001133


In [13]:
#Check absentees
aidx = []
for idx in idx_to_preserve: 
    if idx not in sorted_idx:
        aidx.append(idx)
print len(aidx),len(idx_to_preserve)

0 31165


In [15]:
#Number of words we're double counting for single/double ~ 8k should be OK
subset_w = [vocab[i] for i in sorted_idx]
dblct    = []

for w in subset_w:
    singw=p.singular_noun(w)
    if singw==w[:-1] and singw in subset_w and singw:
        dblct.append(w)
print len(dblct)

7188


In [16]:
data_subset = data.tocsc()[:,sorted_idx].tocsr()

In [17]:
features_subset = [vocab[k] for k in sorted_idx]

In [18]:
features_subset_singular = []
for w in features_subset:
    kk = p.singular_noun(w)
    if kk:
        features_subset_singular.append(kk)
    else:
        features_subset_singular.append(w)

In [19]:
features_subset          = np.array(features_subset)
features_subset_singular = np.array(features_subset_singular)

w2idx = {}
w2idx_sing = {}
for idx,w in enumerate(features_subset.tolist()):
    w2idx[w] = idx
for idx,w in enumerate(features_subset_singular.tolist()):
    w2idx_sing[w] = idx
'hennings' in w2idx

True

In [20]:
#Find all the words they map to
mapIdx = {}
notfound=[]
flist  = features_subset.tolist()
flist_s  = features_subset_singular.tolist()
for w in vocab_emb:
    if w in features_subset:
        #idxlst_vocab = np.where(w==features_subset)[0].tolist()
        #assert len(idxlst_vocab)==1,'nsd'
        idx       = w2idx[w]#flist.index(w)
        mapIdx[w] = idx#idxlst_vocab[0]
    elif w in features_subset_singular:
        #idxlst = np.where(w==features_subset_singular)[0].tolist()
        #assert len(idxlst)==1,'nsd'
        idx       = w2idx_sing[w]#flist_s.index(w)
        mapIdx[w] = idx#idxlst[0]
    else:
        notfound.append(w)
print len(notfound),' not found'

427  not found


In [21]:
#csr_matrix((data, indices, indptr), shape=(3, 3)).toarray()
print data_subset.max(),data_subset.shape
doccts = data_subset.max(1).toarray().squeeze()
docs_keep_idx = np.where(doccts>5)[0]
print docs_keep_idx.shape

data_subset_minlen = data_subset[docs_keep_idx]
print data_subset_minlen.shape, len(features_subset)

2702.0 (3035070, 46379)
(1264480,)
(1264480, 46379) 46379


In [22]:
np.sort(np.array(data_subset_minlen.sum(1)).squeeze())

array([  7.00000000e+00,   7.00000000e+00,   7.00000000e+00, ...,
         1.95980000e+04,   2.04900000e+04,   2.72610000e+04])

In [23]:
#Restrict the documents
np.random.seed(1)
shufidx = np.random.permutation(data_subset_minlen.shape[0])

test_idx    = shufidx[:10000]
valid_idx   = shufidx[10000:12000]
train_idx   = shufidx[12000:]

TRAIN = data_subset_minlen[train_idx]
VALID = data_subset_minlen[valid_idx]
TEST  = data_subset_minlen[test_idx]
print TRAIN.shape, VALID.shape, TEST.shape
print np.sort(np.array(TRAIN.sum(1)).squeeze()).astype(int), np.sort(np.array(VALID.sum(1)).squeeze()), np.sort(np.array(TEST.sum(1)).squeeze())

(1252480, 46379) (2000, 46379) (10000, 46379)
[    7     7     7 ..., 19598 20490 27261] [   22.    22.    22. ...,  4459.  4564.  5338.] [   13.    13.    13. ...,  6789.  9812.  9935.]


In [24]:
cts_train = np.array(TRAIN.sum(0)).squeeze()
for k in mapIdx.values():
    if cts_train[k]<2:
        print features_subset[k],features_subset_singular[k],cts_train[k]

peploes peplo 1.0
tufanos tufano 1.0
bunuels bunuel 1.0
sequer sequer 1.0
direto direto 1.0
marsillach marsillach 1.0
thumpingly thumpingly 1.0
powerment powerment 1.0
pardia pardia 0.0
bierbichler bierbichler 0.0
convertir convertir 1.0
defeatingly defeatingly 1.0
democracie democracie 0.0
ozpetek ozpetek 1.0
yawner yawner 1.0
wertmullers wertmuller 1.0
videostore videostore 0.0
clunkiness clunkines 1.0
achero achero 0.0
espetculo espetculo 1.0
debrauwer debrauwer 1.0
recomend recomend 0.0
payamis payami 1.0
methis methi 0.0
loooong loooong 1.0
slausens slausen 0.0
jacquots jacquot 1.0
andwell andwell 1.0
galaxina galaxina 1.0
taquilla taquilla 1.0
adventues adventue 1.0
koshashvili koshashvili 1.0
freundlichs freundlich 0.0
masterpeice masterpeice 0.0
seldahl seldahl 0.0
hubac hubac 1.0
shmear shmear 1.0
rhapsodize rhapsodize 1.0
ploddingly ploddingly 1.0
condundrum condundrum 1.0
sychowski sychowski 1.0
wynorskis wynorski 1.0
aqueles aquele 1.0
dirigida dirigida 1.0
moviethis moviet

In [25]:
os.system('rm -rf ./wikicorp/data-large.h5 ./wikicorp/misc-large.pkl')
saveSparseHDF5(TRAIN, 'train', './wikicorp/data-large.h5')
saveSparseHDF5(VALID, 'valid', './wikicorp/data-large.h5')
saveSparseHDF5(TEST,  'test' , './wikicorp/data-large.h5')
savePickle([mapIdx,features_subset,features_subset_singular],'./wikicorp/misc-large.pkl')

Saved  3  objects
